In [ ]:
# Train model
import sys
sys.path.append(r'D:\skin_disease_detection\backend\app')
from models import cnn_model

In [ ]:
# Import saved model
from keras.models import load_model
model = load_model(r'D:\skin_disease_detection\backend\app\static\trained_models\skin_disease_model.h5')
print(model)


In [ ]:
import tensorflow as tf

# Print TensorFlow version and device information
print("TensorFlow version:", tf.__version__)

# Check if GPU is available
if tf.config.list_physical_devices('GPU'):
    print("TensorFlow is using GPU")
else:
    print("TensorFlow is using CPU")
